# Effective Python: Chapter 1 - Pythonic Thinking
## Items 5-10

# Item 5: Write Helper Functions Instead of Complex Expressions

Python's pithy syntax makes it easy to write single-line expressions that implement a lot of logic. However, complex expressions can sacrifice readability.

## Example: Parsing URL Query Strings

In [ ]:
from urllib.parse import parse_qs

my_values = parse_qs('red=5&blue=0&green=',
                     keep_blank_values=True)
print(repr(my_values))

In [ ]:
print('Red:     ', my_values.get('red'))
print('Green:   ', my_values.get('green'))
print('Opacity: ', my_values.get('opacity'))

### Problem: Using Boolean Expressions (Too Complex)

In [ ]:
# For query string 'red=5&blue=0&green='

# This line retrieves the 'red' parameter from a query string dictionary
# Breaking down the operation from right to left:
# 1. my_values.get('red', ['']) - Gets the value associated with key 'red'
#    - If 'red' exists, returns its value (typically a list like ['5'])
#    - If 'red' doesn't exist, returns default value [''] (list with empty string)
# 2. [0] - Extracts the first element from the list
#    - Query string parsers typically store values as lists to handle multiple values
#    - For 'red=5', this would be '5' (string)
#    - For missing key, this would be '' (empty string)
# 3. or 0 - Applies boolean short-circuit evaluation
#    - If the left operand is "falsy" (empty string '', None, 0, False), return 0
#    - If the left operand is "truthy" (any non-empty string like '5'), return that value
#    - This provides a default numeric value when the parameter is missing or empty
red = my_values.get('red', [''])[0] or 0

# This line retrieves the 'green' parameter using identical logic
# For query string 'red=5&blue=0&green=':
# 1. my_values.get('green', ['']) returns [''] because green has no value (green=)
# 2. [0] extracts '' (empty string)
# 3. or 0 evaluates '' as falsy, so returns 0
# Result: green = 0
green = my_values.get('green', [''])[0] or 0

# This line retrieves the 'opacity' parameter using identical logic
# For query string 'red=5&blue=0&green=':
# 1. my_values.get('opacity', ['']) returns [''] because 'opacity' key doesn't exist
# 2. [0] extracts '' (empty string) from the default value
# 3. or 0 evaluates '' as falsy, so returns 0
# Result: opacity = 0
opacity = my_values.get('opacity', [''])[0] or 0

# The !r formatting specifier uses repr() to show the "representation" of the value
# This displays values with their type information:
# - Strings show with quotes: 'text'
# - Numbers show without quotes: 0 or 5
# This is useful for debugging to distinguish between string '0' and integer 0
print(f'Red:     {red!r}')
print(f'Green:   {green!r}')
print(f'Opacity: {opacity!r}')

# SUMMARY OF BEHAVIOR:
# ===================
# Given: my_values from query string 'red=5&blue=0&green='
#
# red=5     → my_values['red'] = ['5'] → [0] gives '5' → '5' or 0 = '5'
# green=    → my_values['green'] = [''] → [0] gives '' → '' or 0 = 0
# opacity   → key doesn't exist → default [''] → [0] gives '' → '' or 0 = 0
#
# POTENTIAL ISSUE:
# ================
# This pattern has a subtle bug: the 'or 0' will trigger for ANY falsy value,
# including the string '0' from 'blue=0':
#
# blue=0 → my_values['blue'] = ['0'] → [0] gives '0' → '0' or 0 = '0' (string!)
#
# However, if you later do numeric comparison, '0' (string) might not behave
# as expected. Better pattern would convert to int first:
#
# int(my_values.get('red', ['0'])[0])
#
# Or use more explicit None checking:
#
# value = my_values.get('red', [''])[0]
# red = int(value) if value else 0

This is difficult to read and doesn't do everything needed (converting to int).

In [ ]:
# Even worse - wrapping with int()
red = int(my_values.get('red', [''])[0] or 0)
# This is extremely hard to read!

### Using if/else (Better but verbose)

In [ ]:
# SAFER QUERY STRING PARSING APPROACH
# ====================================
# This is a more explicit and safer alternative to the 'or 0' pattern

# Step 1: Retrieve the value list from the query string dictionary
# my_values.get('green', ['']) attempts to get the 'green' parameter
# - If 'green' exists in the query string, returns its value list (e.g., [''], ['5'], etc.)
# - If 'green' doesn't exist, returns the default value [''] (list containing empty string)
# 
# Unlike the previous pattern, this DOES NOT immediately extract the first element
# Instead, it stores the entire list for subsequent validation
green_str = my_values.get('green', [''])

# Step 2: Check if the first element contains an actual value
# if green_str[0]: tests the truthiness of the first element in the list
# 
# This condition evaluates to True when:
# - green_str[0] is a non-empty string like '5', '100', 'invalid'
# 
# This condition evaluates to False when:
# - green_str[0] is an empty string '' (from 'green=' or missing key)
# - green_str[0] is None (edge case)
# 
# CRITICAL DIFFERENCE from 'or 0' pattern:
# - This checks BEFORE attempting conversion
# - Prevents issues with string '0' being treated as falsy
if green_str[0]:
    # The condition was True, meaning we have a non-empty string value
    # Convert the string to an integer using int()
    # 
    # For 'green=5' → green_str[0] = '5' → int('5') = 5
    # For 'green=42' → green_str[0] = '42' → int('42') = 42
    # 
    # POTENTIAL ERROR: If the value is non-numeric (e.g., 'green=abc'),
    # this will raise ValueError. Production code should use try/except:
    #
    # try:
    #     green = int(green_str[0])
    # except ValueError:
    #     green = 0  # fallback for invalid input
    green = int(green_str[0])
else:
    # The condition was False, meaning the value was empty or missing
    # Assign default value of 0
    # 
    # This handles:
    # - 'green=' (empty value) → green_str[0] = '' → green = 0
    # - Missing 'green' parameter → green_str[0] = '' (from default) → green = 0
    green = 0

# COMPARISON WITH PREVIOUS PATTERN:
# ==================================
#
# OLD PATTERN (problematic):
# green = my_values.get('green', [''])[0] or 0
#
# NEW PATTERN (explicit):
# green_str = my_values.get('green', [''])
# if green_str[0]:
#     green = int(green_str[0])
# else:
#     green = 0
#
# ADVANTAGES OF NEW PATTERN:
# ---------------------------
# 1. Explicit type conversion: int() is called only when a value exists
# 2. Correct handling of '0': String '0' will be converted to integer 0
#    - Old pattern: '0' or 0 = '0' (remains string)
#    - New pattern: int('0') = 0 (properly converts to integer)
# 3. Better error handling potential: Can wrap int() in try/except
# 4. More readable: Clear intent of each step
# 5. Easier to debug: Can inspect green_str before conversion
#
# BEHAVIOR COMPARISON:
# --------------------
# Query: 'green=5'
#   Old: '5' or 0 = '5' (string)
#   New: int('5') = 5 (integer)
#
# Query: 'green=0'
#   Old: '0' or 0 = '0' (string) ← PROBLEM!
#   New: int('0') = 0 (integer) ← CORRECT!
#
# Query: 'green='
#   Old: '' or 0 = 0
#   New: green = 0
#
# Query: (no green parameter)
#   Old: '' or 0 = 0
#   New: green = 0

# PRODUCTION-READY VERSION WITH ERROR HANDLING:
# ==============================================
# green_str = my_values.get('green', [''])
# if green_str[0]:
#     try:
#         green = int(green_str[0])
#     except ValueError:
#         # Handle non-numeric input like 'green=abc'
#         green = 0  # or raise error, log warning, etc.
# else:
#     green = 0

### Solution: Helper Function (Best)

In [ ]:
# REUSABLE QUERY STRING PARAMETER EXTRACTION HELPER FUNCTION
# ===========================================================
# This function encapsulates the safe query string parsing pattern into
# a reusable utility that can be called multiple times with different parameters

def get_first_int(values, key, default=0):
    """
    Extract and convert the first value of a query string parameter to an integer.
    
    This function safely handles query string dictionaries where values are stored
    as lists (common in web frameworks like urllib.parse.parse_qs, Django, Flask).
    
    Parameters:
    -----------
    values : dict
        Dictionary containing query string parameters where each value is a list.
        Example: {'red': ['5'], 'green': [''], 'blue': ['0']}
    
    key : str
        The parameter name to look up in the values dictionary.
        Example: 'red', 'green', 'opacity'
    
    default : int, optional
        The default value to return if the parameter is missing or empty.
        Defaults to 0 if not specified.
    
    Returns:
    --------
    int
        The integer value of the parameter, or the default value if missing/empty.
    
    Examples:
    ---------
    values = {'red': ['5'], 'green': [''], 'blue': ['0']}
    
    get_first_int(values, 'red')      # Returns: 5
    get_first_int(values, 'green')    # Returns: 0 (empty string)
    get_first_int(values, 'opacity')  # Returns: 0 (missing key)
    get_first_int(values, 'opacity', -1)  # Returns: -1 (custom default)
    """
    
    # STEP 1: Retrieve the value list from the dictionary
    # values.get(key, ['']) attempts to get the list associated with 'key'
    # 
    # Behavior:
    # - If 'key' exists: Returns the actual list (e.g., ['5'], [''], ['0'])
    # - If 'key' doesn't exist: Returns default [''] (list with empty string)
    # 
    # Why use [''] as default instead of [] or None?
    # - Ensures found[0] in the next step won't raise IndexError
    # - Empty string [''] allows truthiness check to work correctly
    # - Maintains consistent list structure regardless of key existence
    # 
    # Examples:
    # - values.get('red', ['']) where red='5'  → found = ['5']
    # - values.get('green', ['']) where green= → found = ['']
    # - values.get('opacity', ['']) (missing)  → found = ['']
    found = values.get(key, [''])
    
    # STEP 2: Check if the first element contains a value
    # if found[0]: tests the truthiness of the first element in the list
    # 
    # This evaluates to True when:
    # - found[0] is a non-empty string: '5', '100', '0', 'abc', etc.
    # 
    # This evaluates to False when:
    # - found[0] is an empty string: '' (from 'key=' or missing key)
    # 
    # IMPORTANT: String '0' is truthy!
    # - '0' evaluates to True in boolean context
    # - This correctly handles 'blue=0' → int('0') → 0
    # - Unlike the 'or 0' pattern which treats '0' as falsy
    if found[0]:
        # We have a non-empty string value, attempt integer conversion
        # 
        # int(found[0]) converts the string to an integer
        # - '5' → 5
        # - '100' → 100
        # - '0' → 0 (correctly handles zero)
        # 
        # POTENTIAL ERROR: ValueError if the string is non-numeric
        # - 'abc' would raise ValueError
        # - '3.14' would raise ValueError (use float() for decimals)
        # - '10px' would raise ValueError
        # 
        # Production version should use try/except:
        # try:
        #     return int(found[0])
        # except ValueError:
        #     return default  # or log error, raise custom exception, etc.
        return int(found[0])
    
    # STEP 3: Return default value for empty or missing parameters
    # This executes when found[0] is an empty string ''
    # 
    # Handles two cases:
    # - Parameter exists but has no value: 'key='
    # - Parameter doesn't exist in query string at all
    # 
    # Returns the default parameter value (0 unless caller specified otherwise)
    return default


# USAGE EXAMPLES:
# ===============

# Example 1: Basic usage with query string parser
# from urllib.parse import parse_qs
# query_string = 'red=5&blue=0&green='
# my_values = parse_qs(query_string)
# # Result: {'red': ['5'], 'blue': ['0'], 'green': ['']}
# 
# red = get_first_int(my_values, 'red')        # Returns: 5
# blue = get_first_int(my_values, 'blue')      # Returns: 0
# green = get_first_int(my_values, 'green')    # Returns: 0 (empty value)
# opacity = get_first_int(my_values, 'opacity') # Returns: 0 (missing key)

# Example 2: Custom default value
# red = get_first_int(my_values, 'red', default=255)
# # If 'red' is missing or empty, returns 255 instead of 0

# Example 3: Replacing repetitive code
# Without helper function (repetitive):
# red = my_values.get('red', [''])[0] or 0
# green = my_values.get('green', [''])[0] or 0
# blue = my_values.get('blue', [''])[0] or 0  # BUG: '0' stays as string!
# opacity = my_values.get('opacity', [''])[0] or 0
#
# With helper function (clean and correct):
# red = get_first_int(my_values, 'red')
# green = get_first_int(my_values, 'green')
# blue = get_first_int(my_values, 'blue')      # CORRECT: 0 as integer
# opacity = get_first_int(my_values, 'opacity')


# ADVANTAGES OF THIS HELPER FUNCTION:
# ====================================
# 1. DRY Principle: Eliminates repetitive query string parsing code
# 2. Type Safety: Always returns an integer (or default), never mixed types
# 3. Correct Zero Handling: String '0' properly converts to integer 0
# 4. Configurable Defaults: Caller can specify custom default values
# 5. Readability: Intent is clear from function name
# 6. Maintainability: Bug fixes apply to all usages
# 7. Testability: Single function to unit test instead of scattered code
# 8. Extensibility: Easy to add error handling, logging, validation


# PRODUCTION-READY VERSION WITH ERROR HANDLING:
# ==============================================
def get_first_int_safe(values, key, default=0):
    """Production version with error handling and validation."""
    found = values.get(key, [''])
    if found[0]:
        try:
            return int(found[0])
        except ValueError:
            # Handle non-numeric input gracefully
            # Options: return default, log warning, raise custom exception
            import logging
            logging.warning(f"Invalid integer value for '{key}': {found[0]!r}")
            return default
    return default


# ALTERNATIVE: Handle multiple values (if query has 'color=red&color=blue')
def get_all_ints(values, key, default=None):
    """
    Extract all values for a parameter and convert to integers.
    
    Returns:
        list of int: All valid integer values, or [default] if none found
    """
    found = values.get(key, [])
    if not found:
        return [default] if default is not None else []
    
    result = []
    for value in found:
        if value:
            try:
                result.append(int(value))
            except ValueError:
                continue  # Skip invalid values
    
    return result if result else ([default] if default is not None else [])


# COMPARISON WITH COMMON PATTERNS:
# =================================
#
# Pattern 1: Direct indexing (UNSAFE - can raise KeyError/IndexError)
# value = int(my_values['red'][0])
#
# Pattern 2: get() with 'or' (BUGGY - treats '0' as falsy)
# value = my_values.get('red', [''])[0] or 0
#
# Pattern 3: Manual if/else (VERBOSE - repeats for each parameter)
# found = my_values.get('red', [''])
# if found[0]:
#     value = int(found[0])
# else:
#     value = 0
#
# Pattern 4: Helper function (RECOMMENDED - clean, safe, reusable)
# value = get_first_int(my_values, 'red')

In [ ]:
# Much clearer!
green = get_first_int(my_values, 'green')
print(f'Green: {green}')

### Things to Remember

- Python's syntax makes it easy to write single-line expressions that are overly complicated and difficult to read
- Move complex expressions into helper functions, especially if you need to use the same logic repeatedly
- An if/else expression provides a more readable alternative to using Boolean operators or and and in expressions

# Item 6: Prefer Multiple Assignment Unpacking Over Indexing

Python has built-in tuple type for creating immutable, ordered sequences of values.

In [1]:
snack_calories = {
    'chips': 140,
    'popcorn': 80,
    'nuts': 190,
}
items = tuple(snack_calories.items())
print(items)

(('chips', 140), ('popcorn', 80), ('nuts', 190))


### Accessing with Indexes (Traditional)

In [2]:
item = ('Peanut butter', 'Jelly')
first = item[0]
second = item[1]
print(first, 'and', second)

Peanut butter and Jelly


### Using Unpacking (Pythonic)

In [4]:
item = ('Peanut butter', 'Jelly')
first, second = item  # Unpacking
print(first, 'and', second)
print(first)

Peanut butter and Jelly
Peanut butter


### Unpacking with Nested Structures

In [ ]:
favorite_snacks = {
    'salty': ('pretzels', 100),
    'sweet': ('cookies', 180),
    'veggie': ('carrots', 20),
}

((type1, (name1, cals1)),
 (type2, (name2, cals2)),
 (type3, (name3, cals3))) = favorite_snacks.items()

print(f'Favorite {type1} is {name1} with {cals1} calories')
print(f'Favorite {type2} is {name2} with {cals2} calories')
print(f'Favorite {type3} is {name3} with {cals3} calories')

### Swapping Values

In [ ]:
# BUBBLE SORT IMPLEMENTATION WITH TUPLE UNPACKING
# ================================================
# This is a classic sorting algorithm that repeatedly steps through the list,
# compares adjacent elements, and swaps them if they're in the wrong order.
# The algorithm gets its name because smaller elements "bubble" to the top.

def bubble_sort(a):
    """
    Sort a list in-place using the bubble sort algorithm.
    
    Time Complexity: O(n²) - quadratic time for average and worst case
    Space Complexity: O(1) - sorts in-place, no extra memory needed
    
    Parameters:
    -----------
    a : list
        The list to sort (modified in-place). Works with any comparable types.
    
    Returns:
    --------
    None
        Modifies the input list directly (in-place sorting)
    
    Note:
    -----
    This is an educational implementation. For production code, use:
    - list.sort() for in-place sorting
    - sorted(list) for creating a new sorted list
    Both use Timsort (O(n log n)) which is much faster.
    """
    
    # OUTER LOOP: Controls the number of passes through the list
    # ==========================================================
    # for _ in range(len(a)):
    # 
    # Purpose: Ensures every element has a chance to bubble to its correct position
    # 
    # Loop iterations: len(a) times
    # - For a 4-element list: runs 4 complete passes
    # - Underscore (_) indicates we don't use the loop variable
    # 
    # Why len(a) passes?
    # - After each pass, at least one element is guaranteed in correct position
    # - After n passes, all n elements must be sorted
    # - This is actually more passes than necessary (could optimize to len(a)-1)
    # 
    # Example with [3, 1, 2]:
    # Pass 1: [1, 2, 3] - largest element (3) bubbles to end
    # Pass 2: [1, 2, 3] - second largest (2) is already correct
    # Pass 3: [1, 2, 3] - no changes (already sorted, but we still check)
    for _ in range(len(a)):
        
        # INNER LOOP: Performs one complete pass through the list
        # =======================================================
        # for i in range(1, len(a)):
        # 
        # Purpose: Compare and swap adjacent elements during one pass
        # 
        # Loop range: 1 to len(a)-1 (inclusive)
        # - Starts at index 1 (not 0) because we compare a[i] with a[i-1]
        # - If we started at 0, a[i-1] would be a[-1] (last element), wrong!
        # 
        # Why start at 1?
        # - We need to access the previous element (i-1)
        # - i=1 gives us pairs: (a[0], a[1]), then (a[1], a[2]), etc.
        # 
        # Example with [3, 1, 2, 4]:
        # i=1: Compare a[0]=3 and a[1]=1
        # i=2: Compare a[1]=? and a[2]=2  (a[1] may have changed from swap)
        # i=3: Compare a[2]=? and a[3]=4
        for i in range(1, len(a)):
            
            # COMPARISON: Check if adjacent elements are out of order
            # ========================================================
            # if a[i] < a[i-1]:
            # 
            # Purpose: Determine if a swap is needed
            # 
            # Logic: If current element is smaller than previous element,
            #        they are out of order and need to be swapped
            # 
            # Comparison direction:
            # - a[i] < a[i-1] sorts in ASCENDING order (smallest to largest)
            # - For descending: use a[i] > a[i-1]
            # 
            # Example comparisons:
            # ['pretzels', 'carrots'] → 'carrots' < 'pretzels' → True (swap)
            # ['bacon', 'carrots'] → 'carrots' > 'bacon' → False (no swap)
            # [3, 1] → 1 < 3 → True (swap)
            # [1, 3] → 3 < 1 → False (no swap)
            if a[i] < a[i-1]:
                
                # SWAP: Exchange positions of two adjacent elements
                # ==================================================
                # a[i-1], a[i] = a[i], a[i-1]
                # 
                # This is Python's tuple unpacking swap idiom
                # It's equivalent to but cleaner than:
                # temp = a[i-1]
                # a[i-1] = a[i]
                # a[i] = temp
                # 
                # How tuple unpacking works:
                # 1. Right side creates a tuple: (a[i], a[i-1])
                # 2. Left side unpacks tuple into variables: a[i-1], a[i]
                # 3. Assignment happens simultaneously (atomic operation)
                # 
                # Example with ['carrots', 'pretzels'] at indices [0, 1]:
                # Before: a[0]='pretzels', a[1]='carrots'
                # Right side evaluates: ('carrots', 'pretzels')
                # Assignment: a[0]='carrots', a[1]='pretzels'
                # After: a[0]='carrots', a[1]='pretzels'
                # 
                # Why this works without temporary variable:
                # - Python evaluates entire right side before any assignment
                # - Values are captured in tuple before variables change
                # - No risk of overwriting values prematurely
                # 
                # Performance note:
                # - Tuple creation has minimal overhead
                # - More readable than traditional three-line swap
                # - Pythonic idiom preferred by Python community
                a[i-1], a[i] = a[i], a[i-1]  # Swap with unpacking


# DEMONSTRATION: Sorting a list of strings
# =========================================

# Initial list of food items (unsorted)
# String comparison uses lexicographic (alphabetical) ordering
names = ['pretzels', 'carrots', 'arugula', 'bacon']

# Call bubble_sort to sort the list in-place
# After this call, 'names' will be modified to ['arugula', 'bacon', 'carrots', 'pretzels']
bubble_sort(names)

# Print the sorted result
# Expected output: ['arugula', 'bacon', 'carrots', 'pretzels']
print(names)


# DETAILED EXECUTION TRACE FOR ['pretzels', 'carrots', 'arugula', 'bacon']:
# ==========================================================================

# OUTER LOOP ITERATION 1 (first complete pass):
# ----------------------------------------------
# Initial: ['pretzels', 'carrots', 'arugula', 'bacon']
#
# i=1: Compare 'carrots' < 'pretzels' → True → SWAP
#      Result: ['carrots', 'pretzels', 'arugula', 'bacon']
#
# i=2: Compare 'arugula' < 'pretzels' → True → SWAP
#      Result: ['carrots', 'arugula', 'pretzels', 'bacon']
#
# i=3: Compare 'bacon' < 'pretzels' → True → SWAP
#      Result: ['carrots', 'arugula', 'bacon', 'pretzels']
#
# After Pass 1: ['carrots', 'arugula', 'bacon', 'pretzels']
# Note: 'pretzels' (largest) has bubbled to the end

# OUTER LOOP ITERATION 2 (second complete pass):
# -----------------------------------------------
# Initial: ['carrots', 'arugula', 'bacon', 'pretzels']
#
# i=1: Compare 'arugula' < 'carrots' → True → SWAP
#      Result: ['arugula', 'carrots', 'bacon', 'pretzels']
#
# i=2: Compare 'bacon' < 'carrots' → True → SWAP
#      Result: ['arugula', 'bacon', 'carrots', 'pretzels']
#
# i=3: Compare 'pretzels' < 'carrots' → False → NO SWAP
#      Result: ['arugula', 'bacon', 'carrots', 'pretzels']
#
# After Pass 2: ['arugula', 'bacon', 'carrots', 'pretzels']
# Note: 'carrots' is now in correct position

# OUTER LOOP ITERATION 3 (third complete pass):
# ----------------------------------------------
# Initial: ['arugula', 'bacon', 'carrots', 'pretzels']
#
# i=1: Compare 'bacon' < 'arugula' → False → NO SWAP
# i=2: Compare 'carrots' < 'bacon' → False → NO SWAP
# i=3: Compare 'pretzels' < 'carrots' → False → NO SWAP
#
# After Pass 3: ['arugula', 'bacon', 'carrots', 'pretzels']
# Note: List is sorted, but algorithm doesn't know to stop early

# OUTER LOOP ITERATION 4 (fourth complete pass):
# -----------------------------------------------
# Initial: ['arugula', 'bacon', 'carrots', 'pretzels']
# No swaps occur - list already sorted
# After Pass 4: ['arugula', 'bacon', 'carrots', 'pretzels']

# FINAL RESULT: ['arugula', 'bacon', 'carrots', 'pretzels']


# ALGORITHM CHARACTERISTICS:
# ==========================

# Stability: STABLE
# - Equal elements maintain their relative order
# - Example: [3a, 3b, 1] → [1, 3a, 3b] (3a stays before 3b)

# In-Place: YES
# - Sorts the original list without creating a new one
# - Space complexity O(1) - only uses a constant amount of extra memory

# Adaptive: NO (in this implementation)
# - Takes the same time even if the list is already sorted
# - Could be optimized to detect when no swaps occur and exit early

# Best Case: O(n²) in this implementation
# - Could be O(n) with optimization (early exit when sorted)

# Average Case: O(n²)
# - Roughly n²/2 comparisons on average

# Worst Case: O(n²)
# - Occurs when list is reverse sorted
# - Maximum number of swaps needed


# OPTIMIZED VERSION WITH EARLY EXIT:
# ===================================
def bubble_sort_optimized(a):
    """
    Optimized bubble sort that exits early if no swaps occur.
    Best case improves to O(n) for already-sorted lists.
    """
    for _ in range(len(a)):
        # Flag to track if any swaps occurred in this pass
        swapped = False
        
        for i in range(1, len(a)):
            if a[i] < a[i-1]:
                a[i-1], a[i] = a[i], a[i-1]
                swapped = True  # Mark that we made a swap
        
        # If no swaps occurred, list is sorted - exit early
        if not swapped:
            break


# COMPARISON WITH PYTHON'S BUILT-IN SORT:
# ========================================

# Bubble sort (this implementation):
# names = ['pretzels', 'carrots', 'arugula', 'bacon']
# bubble_sort(names)
# Time: O(n²), Space: O(1)

# Python's built-in sort (Timsort algorithm):
# names = ['pretzels', 'carrots', 'arugula', 'bacon']
# names.sort()
# Time: O(n log n), Space: O(n)

# Python's sorted function (creates new list):
# names = ['pretzels', 'carrots', 'arugula', 'bacon']
# sorted_names = sorted(names)
# Time: O(n log n), Space: O(n)

# For production code: ALWAYS use Python's built-in sorting!
# Bubble sort is primarily for educational purposes.


# WHEN TO USE BUBBLE SORT (RARELY):
# ==================================
# 1. Educational purposes - learning sorting algorithms
# 2. Very small datasets (< 10 elements) where simplicity matters more than speed
# 3. Nearly sorted data with early-exit optimization
# 4. Embedded systems with severe memory constraints (in-place sorting)

# WHEN NOT TO USE (MOST CASES):
# ==============================
# - Any dataset larger than a few dozen elements
# - Performance-critical applications
# - When you have access to better algorithms (you almost always do)
# - Production code (use list.sort() or sorted() instead)

### Unpacking in Loops

In [ ]:
snacks = [('bacon', 350), ('donut', 240), ('muffin', 190)]

# Without unpacking (noisy)
for i in range(len(snacks)):
    item = snacks[i]
    name = item[0]
    calories = item[1]
    print(f'#{i+1}: {name} has {calories} calories')

In [ ]:
#more pythonic
# With unpacking and enumerate (Pythonic)
for rank, (name, calories) in enumerate(snacks, 1):
    print(f'#{rank}: {name} has {calories} calories')

### Things to Remember

- Python has special syntax called unpacking for assigning multiple values in a single statement
- Unpacking is generalized in Python and can be applied to any iterable, including many levels of iterables within iterables
- Reduce visual noise and increase code clarity by using unpacking to avoid explicitly indexing into sequences

# Item 7: Prefer enumerate Over range

The range built-in function is useful for loops that iterate over a set of integers.

In [5]:
from random import randint

random_bits = 0
for i in range(32):
    if randint(0, 1):
        # This line performs three operations:
        # 1. (1 << i): Left-shifts the bit 1 by i positions
        #    - When i=0: creates 0b00000001 (sets bit 0)
        #    - When i=1: creates 0b00000010 (sets bit 1)
        #    - When i=2: creates 0b00000100 (sets bit 2)
        #    - etc., creating a "mask" with only bit i set to 1
        #
        # 2. (|): Bitwise OR operation
        #    - Combines random_bits with the mask
        #    - OR truth: 0|0=0, 0|1=1, 1|0=1, 1|1=1
        #    - Result: sets bit i to 1, preserves all other bits
        #
        # 3. (|=): Compound assignment
        #    - Equivalent to: random_bits = random_bits | (1 << i)
        #    - Updates random_bits with the new bit pattern
        #
        # Net effect: Sets the bit at position i to 1 in random_bits
        # without affecting any previously set bits
        random_bits |= 1 << i
        
print(bin(random_bits))

0b111100111010100001100001001010


### Iterating Over Lists Directly

In [6]:
flavor_list = ['vanilla', 'chocolate', 'pecan', 'strawberry']
for flavor in flavor_list:
    print(f'{flavor} is delicious')

vanilla is delicious
chocolate is delicious
pecan is delicious
strawberry is delicious


### Problem: When You Need the Index

In [7]:
# Using range (clumsy)
for i in range(len(flavor_list)):
    flavor = flavor_list[i]
    print(f'{i + 1}: {flavor}')

1: vanilla
2: chocolate
3: pecan
4: strawberry


### Solution: Use enumerate

In [ ]:
# enumerate wraps any iterator with a lazy generator
# that yields (index, value) tuples on each iteration
it = enumerate(flavor_list)

# next() retrieves the subsequent tuple from the generator
# producing (0, first_element), then (1, second_element), etc.
print(next(it))
print(next(it))

(0, 'vanilla')
(1, 'chocolate')


In [9]:
# Much clearer with unpacking
for i, flavor in enumerate(flavor_list):
    print(f'{i + 1}: {flavor}')

1: vanilla
2: chocolate
3: pecan
4: strawberry


### Specifying Start Number

In [10]:
# Start counting from 1 instead of 0
for i, flavor in enumerate(flavor_list, 1):
    print(f'{i}: {flavor}')

1: vanilla
2: chocolate
3: pecan
4: strawberry


### Things to Remember

- enumerate provides concise syntax for looping over an iterator and getting the index of each item from the iterator as you go
- Prefer enumerate instead of looping over a range and indexing into a sequence
- You can supply a second parameter to enumerate to specify the number from which to begin counting (zero is the default)

# Item 8: Use zip to Process Iterators in Parallel

Often you find yourself with many lists of related objects.

In [ ]:
names = ['Cecilia', 'Lise', 'Marie']

# List comprehension: iterates through names, applying len() to each element
# Creates new list containing the length of each name string
counts = [len(n) 
          for n in names]

print(counts)

[7, 4, 5]


### Problem: Iterating with range and Indexing

In [ ]:
longest_name = None
max_count = 0

for i in range(len(names)):
    count = counts[i]
    if count > max_count:
        longest_name = names[i]
        max_count = count

print(longest_name)

This is visually noisy. Indexing into arrays happens twice.

### Solution: Use zip

In [12]:
# zip wraps two or more iterators with a lazy generator
names = ['Cecilia', 'Lise', 'Marie']

longest_name = None
max_count = 0

for name, count in zip(names, counts):
    if count > max_count:
        longest_name = name
        max_count = count

print(longest_name)

Cecilia


### Warning: zip Truncates to Shortest Iterator

In [ ]:
names.append('Rosalind')
for name, count in zip(names, counts):
    print(name)
# 'Rosalind' is missing!

### Solution: Use zip_longest for Unequal Lengths

In [ ]:
import itertools

for name, count in itertools.zip_longest(names, counts):
    print(f'{name}: {count}')

zip_longest replaces missing values with None (or a fillvalue you specify).

### Things to Remember

- The zip built-in function can be used to iterate over multiple iterators in parallel
- zip creates a lazy generator that produces tuples, so it can be used on infinitely long inputs
- zip truncates its output silently to the shortest iterator if you supply it with iterators of different lengths
- Use the zip_longest function from the itertools built-in module if you want to use zip on iterators of unequal lengths without truncation

# Item 9: Avoid else Blocks After for and while Loops

Python loops have an extra feature: You can put an else block immediately after a loop's repeated interior block.

In [14]:
for i in range(3):
    print('Loop', i)
else:
    print('Else block!')

Loop 0
Loop 1
Loop 2
Else block!


Surprisingly, the else block runs immediately after the loop finishes.

### Behavior with break

In [13]:
for i in range(3):
    print('Loop', i)
    if i == 1:
        break
else:
    print('Else block!')
# Else block is skipped!

Loop 0
Loop 1


### Behavior with Empty Sequences

In [15]:
for x in []:
    print('Never runs')
else:
    print('For Else block!')

For Else block!


In [16]:
while False:
    print('Never runs')
else:
    print('While Else block!')

While Else block!


### Example: Finding Coprime Numbers

In [ ]:
a = 4
b = 9

for i in range(2, min(a, b) + 1):
    print('Testing', i)
    if a % i == 0 and b % i == 0:
        print('Not coprime')
        break
else:
    print('Coprime')

### Better: Use Helper Functions

In [ ]:
# First approach: Return early
def coprime(a, b):
    # Iterate from 2 to the smaller number (inclusive via +1 for range's exclusive end)
    # Only test up to min(a,b) since no divisor can exceed the smaller value
    for i in range(2, min(a, b) + 1):
        # If i divides both a and b evenly, they share a common factor > 1
        # Immediately return False since they're not co-prime
        if a % i == 0 and b % i == 0:
            return False
    
    # No common factors found in range [2, min(a,b)]
    # Therefore gcd(a,b) = 1 and numbers are co-prime
    return True

assert coprime(4, 9)
assert not coprime(3, 6)
print('Tests passed!')

In [ ]:
# Second approach: Use result variable
def coprime_alternate(a, b):
    is_coprime = True
    for i in range(2, min(a, b) + 1):
        if a % i == 0 and b % i == 0:
            is_coprime = False
            break
    return is_coprime

assert coprime_alternate(4, 9)
assert not coprime_alternate(3, 6)
print('Tests passed!')

### Things to Remember

- Python has special syntax that allows else blocks to immediately follow for and while loop interior blocks
- The else block after a loop runs only if the loop body did not encounter a break statement
- Avoid using else blocks after loops because their behavior isn't intuitive and can be confusing

# Item 10: Prevent Repetition with Assignment Expressions

Assignment expressions (the walrus operator :=) were introduced in Python 3.8 to solve code duplication problems.

In [19]:
fresh_fruit = {
    'apple': 10,
    'banana': 8,
    'lemon': 5,
}

### Problem: Repetitive Code

In [20]:
def make_lemonade(count):
    print(f'Making {count} lemonades')

def out_of_stock():
    print('Out of stock!')

# Traditional approach
count = fresh_fruit.get('lemon', 0)
if count:
    make_lemonade(count)
else:
    out_of_stock()

Making 5 lemonades


### Solution: Walrus Operator

In [21]:
# Assignment expression (:= walrus operator) assigns AND evaluates in one step
# Gets 'lemon' count from dict (defaults to 0 if key missing), assigns to count variable
if count := fresh_fruit.get('lemon', 0):
    # If count is truthy (non-zero), lemons are available
    # count variable is now accessible in this scope with the retrieved value
    make_lemonade(count)
else:
    # If count is 0 (falsy), no lemons in inventory
    # Executes out-of-stock logic
    out_of_stock()

Making 5 lemonades


In [23]:
# Example 1: Basic walrus operator demonstration
# WITHOUT walrus operator (traditional approach)
# Requires two lines: assignment then condition check
def get_value():
    return 42

value = get_value()
if value:
    print(f"Traditional: {value}")

# WITH walrus operator (compact approach)
# Assignment and condition check happen in single expression
if value := get_value():
    print(f"Walrus: {value}")


# Example 2: List processing
# WITHOUT walrus - check length separately
my_list = [1, 2, 3, 4, 5]
length = len(my_list)
if length > 3:
    print(f"Traditional: List has {length} items")

# WITH walrus - assign and check in one line
if (length := len(my_list)) > 3:
    print(f"Walrus: List has {length} items")


# Example 3: Dictionary lookup
# WITHOUT walrus
inventory = {'apples': 10, 'bananas': 0, 'oranges': 5}
apple_count = inventory.get('apples', 0)
if apple_count:
    print(f"Traditional: We have {apple_count} apples")

# WITH walrus
if apple_count := inventory.get('apples', 0):
    print(f"Walrus: We have {apple_count} apples")

Traditional: 42
Walrus: 42
Traditional: List has 5 items
Walrus: List has 5 items
Traditional: We have 10 apples
Walrus: We have 10 apples


### Assignment Expression in Comparisons

In [ ]:
def make_cider(count):
    print(f'Making {count} ciders')

# Traditional
count = fresh_fruit.get('apple', 0)
if count >= 4:
    make_cider(count)
else:
    out_of_stock()

In [ ]:
# With walrus (note the parentheses)
if (count := fresh_fruit.get('apple', 0)) >= 4:
    make_cider(count)
else:
    out_of_stock()

### Switch/Case Pattern

In [ ]:
def make_smoothies(count):
    print(f'Making smoothies')
    
def slice_bananas(count):
    return count * 4

# Traditional (deeply nested)
count = fresh_fruit.get('banana', 0)
if count >= 2:
    pieces = slice_bananas(count)
    to_enjoy = make_smoothies(pieces)
else:
    count = fresh_fruit.get('apple', 0)
    if count >= 4:
        to_enjoy = make_cider(count)
    else:
        count = fresh_fruit.get('lemon', 0)
        if count:
            to_enjoy = make_lemonade(count)
        else:
            to_enjoy = 'Nothing'

In [ ]:
# With walrus (elegant)
if (count := fresh_fruit.get('banana', 0)) >= 2:
    pieces = slice_bananas(count)
    to_enjoy = make_smoothies(pieces)
elif (count := fresh_fruit.get('apple', 0)) >= 4:
    to_enjoy = make_cider(count)
elif count := fresh_fruit.get('lemon', 0):
    to_enjoy = make_lemonade(count)
else:
    to_enjoy = 'Nothing'
    
print(f'Result: {to_enjoy}')

### Do/While Loop Pattern

In [ ]:
def pick_fruit():
    # Simulate picking fruit (returns empty dict to stop)
    import random
    if random.random() < 0.7:
        return {'apple': 1}
    return {}

def make_juice(fruit, count):
    return [f'{fruit}_juice']

# Traditional (loop-and-a-half)
bottles = []
while True:
    fresh_fruit = pick_fruit()
    if not fresh_fruit:
        break
    for fruit, count in fresh_fruit.items():
        batch = make_juice(fruit, count)
        bottles.extend(batch)
        
print(f'Made {len(bottles)} bottles')

In [ ]:
# With walrus (cleaner)
bottles = []
while fresh_fruit := pick_fruit():
    for fruit, count in fresh_fruit.items():
        batch = make_juice(fruit, count)
        bottles.extend(batch)
        
print(f'Made {len(bottles)} bottles')

### Things to Remember

- Assignment expressions use the walrus operator (:=) to both assign and evaluate variable names in a single expression, thus reducing repetition
- When an assignment expression is a subexpression of a larger expression, it must be surrounded with parentheses
- Although switch/case statements and do/while loops are not available in Python, their functionality can be emulated much more clearly by using assignment expressions